In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-08-07 16:24:21 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-08-07 16:24:21 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-08-07 16:24:21 __main__ INFO     transformers.__version__='4.54.1'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-08-07 16:24:24 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-07 16:24:24 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-08-07 16:24:24 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-08-07 16:24:24 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-08-07 16:24:25 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-08-07 16:24:25 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-08-07 16:24:25 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-08-07 16:24:25 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-08-07 16:24:25 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.3-70B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

2025-08-07 16:25:12 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-08-07 16:25:12 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-08-07 16:25:12 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask

select_prof = SelectOneTask.load(
    path=os.path.join(env_utils.DEFAULT_DATA_DIR, "selection", "profession.json")
)

print(select_prof)

SelectOneTask: (profession of a famous person)
Categories: actor(20), singer(20), comedian(20), director(20), basketball player(20), football player(20), soccer player(20), tennis player(20), golfer(20), boxer(20), news anchor(20), journalist(20), author(20), fashion designer(20), entrepreneur(19), politician(20)



In [6]:
sample = select_prof.get_random_sample(
    mt = mt,
    prompt_template_idx=2,
    option_style="numbered",
    category="actor"
)

print(sample)

2025-08-07 16:25:25 src.selection.data ERROR    Sample = Will Smith -> Angelina Jolie (0): ['Angelina Jolie', 'Robert Lewandowski', 'James Harden', 'Jim Carrey', 'Steven Spielberg', 'Brandon Sanderson']
    Top prediction " The"[578] (p=0.188, logit=17.250) does not match the object Angelina Jolie[23950, " Angel"].
    Retry count: 1. Retrying ...
    
2025-08-07 16:25:25 src.selection.data ERROR    Sample = Chris Hemsworth -> Al Pacino (1): ['Vinícius Júnior', 'Al Pacino', 'Rihanna', 'Kevin Durant', 'Robin Roberts', 'Phil Mickelson']
    Top prediction " "[220] (p=0.428, logit=19.125) does not match the object Al Pacino[1708, " Al"].
    Retry count: 2. Retrying ...
    
Chris Hemsworth -> Morgan Freeman (4): ['Canelo Álvarez', 'Bill Burr', 'Son Heung-min', 'Billie Eilish', 'Morgan Freeman', 'Jim Acosta']


In [7]:
# sample.prompt_template = select_prof.prompt_templates[3]

print(f'"{sample.prompt()}"', ">>", sample.obj)

"Which person from the following list is by profession a actor?
1. Canelo Álvarez
2. Bill Burr
3. Son Heung-min
4. Billie Eilish
5. Morgan Freeman
6. Jim Acosta
Answer:" >> Morgan Freeman


In [8]:
print(sample.prompt(option_style="single_line"))

Which person from the following list is by profession a actor?
Options: Canelo Álvarez, Bill Burr, Son Heung-min, Billie Eilish, Morgan Freeman, Jim Acosta.
Answer:


In [9]:
from src.functional import generate_with_patch

gen = generate_with_patch(
    mt = mt,
    inputs = sample.prompt(),
    max_new_tokens=20,
    do_sample=False,
    remove_prefix=True
)[0]
print(f'"{gen}"', ">>", sample.obj)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


" Morgan Freeman is an actor by profession. He is a renowned American actor, director, and narrator known" >> Morgan Freeman


In [10]:
from src.experiments.utils import (
    get_patches_to_verify_independent_enrichment,
    verify_head_patterns,
)

HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

attn_pattern = verify_head_patterns(
    prompt = sample.prompt(),
    options = sample.options,
    pivot = sample.subj,
    mt = mt,
    heads = HEADS
)

0 patches to ablate possible answer information from options
2025-08-07 16:25:33 src.experiments.utils DEBUG    Predictions: ['" Morgan"[23809] (p=0.324, logit=19.250)', '" "[220] (p=0.285, logit=19.125)', '" The"[578] (p=0.135, logit=18.375)', '" Bill"[8766] (p=0.082, logit=17.875)', '" ("[320] (p=0.027, logit=16.750)']
2025-08-07 16:25:33 src.experiments.utils INFO     Combined attention matrix for all heads


In [11]:
select_prof.category_wise_examples.keys()

dict_keys(['actor', 'singer', 'comedian', 'director', 'basketball player', 'football player', 'soccer player', 'tennis player', 'golfer', 'boxer', 'news anchor', 'journalist', 'author', 'fashion designer', 'entrepreneur', 'politician'])

In [12]:
import copy
import random
from src.selection.utils import KeyedSet, get_first_token_id
from src.selection.data import SelectionSample
from src.functional import predict_next_token

######################################################################
N_DISTRACTORS = 5
WINDOW_SPEC = {
    mt.layer_name_format: 1,
    mt.mlp_module_name_format: 9,
    mt.attn_module_name_format: 9,
}
module_name_format = mt.layer_name_format
# module_name_format = mt.mlp_module_name_format
# module_name_format = mt.attn_module_name_format

######################################################################


def get_counterfactual_samples_on_pivot_entity(
    task: SelectOneTask = select_prof,
    patch_category: str | None = None,
    clean_category: str | None = None,
    shuffle_clean_options: bool = False,
    prompt_template_idx=2,
    option_style="numbered",
    filter_by_lm_prediction: bool = True,
):
    categories = list(task.category_wise_examples.keys())
    if patch_category is None:
        patch_category = random.choice(categories)

    patch_subj, patch_obj = random.sample(
        task.category_wise_examples[patch_category], 2
    )
    logger.info(
        f"Patch category: {patch_category}, subject: {patch_subj}, object: {patch_obj}"
    )

    if clean_category is None:
        clean_category = random.choice(list(set(categories) - {patch_category}))

    clean_options = task.category_wise_examples[clean_category]
    random.shuffle(clean_options)

    clean_subj, clean_obj = random.sample(
        (KeyedSet(clean_options, mt.tokenizer) - KeyedSet([patch_obj], mt.tokenizer)).values, 2
    )
    logger.info(
        f"Clean category: {clean_category}, subject: {clean_subj}, object: {clean_obj}"
    )

    distractors = []
    other_categories = random.sample(
        list(set(categories) - {patch_category, clean_category}),
        k=N_DISTRACTORS - 1,
    )

    for other_category in other_categories:
        other_examples = task.category_wise_examples[other_category]
        random.shuffle(other_examples)
        other_examples = KeyedSet(other_examples, mt.tokenizer)
        distractors.append(
            random.choice(
                (
                    other_examples
                    - KeyedSet(
                        [patch_obj, clean_obj] + distractors, tokenizer=mt.tokenizer
                    )
                ).values
            )
        )

    patch_options = [patch_obj, clean_obj] + distractors
    random.shuffle(patch_options)
    patch_obj_idx = patch_options.index(patch_obj)
    logger.info(f"{patch_obj_idx=} | {patch_options}")

    clean_options = copy.deepcopy(patch_options)

    if shuffle_clean_options:
        # Useful for the pointer experiments
        while (
            clean_options.index(clean_obj) == patch_obj_idx
            or clean_options.index(patch_obj) == patch_obj_idx
        ):
            random.shuffle(clean_options)

    clean_obj_idx = clean_options.index(clean_obj)

    logger.info(f"{clean_obj_idx=} | {clean_options}")

    kwargs = dict(
        prompt_template= task.prompt_templates[prompt_template_idx],
        default_option_style=option_style,
    )

    patch_sample = SelectionSample(
        subj=patch_subj,
        obj=patch_obj,
        obj_idx=patch_obj_idx,
        obj_token_id=get_first_token_id(patch_obj, mt.tokenizer, prefix=" "),
        options=patch_options,
        category=patch_category,
        **kwargs,
    )
    clean_sample = SelectionSample(
        subj=clean_subj,
        obj=clean_obj,
        obj_idx=clean_obj_idx,
        obj_token_id=get_first_token_id(clean_obj, mt.tokenizer, prefix=" "),
        options=clean_options,
        category=clean_category,
        **kwargs,
    )

    if filter_by_lm_prediction:
        for sample in [patch_sample, clean_sample]:
            pred = predict_next_token(
                mt=mt,
                inputs=sample.prompt(),
            )[0]
            logger.info(f"{sample.subj} -> {sample.obj} | pred={[str(p) for p in pred]}")
            if pred[0].token_id != sample.obj_token_id:
                logger.error(
                    f'Prediction mismatch: {pred[0].token_id}["{mt.tokenizer.decode(pred[0].token_id)}"] != {sample.obj_token_id}["{mt.tokenizer.decode(sample.obj_token_id)}"]'
                )
                return get_counterfactual_samples_on_pivot_entity(
                    task=task,
                    patch_category=patch_category,
                    clean_category=clean_category,
                    shuffle_clean_options=shuffle_clean_options,
                    prompt_template_idx=prompt_template_idx,
                    option_style=option_style,
                    filter_by_lm_prediction=filter_by_lm_prediction,
                )
            sample.prediction = pred

    return patch_sample, clean_sample

In [13]:
patch_sample, clean_sample = get_counterfactual_samples_on_pivot_entity(
    patch_category="politician",
    clean_category="actor",
    filter_by_lm_prediction=False,
    prompt_template_idx=3
)

2025-08-07 16:25:37 __main__ INFO     Patch category: politician, subject: Ron DeSantis, object: Ted Cruz


2025-08-07 16:25:37 __main__ INFO     Clean category: actor, subject: Will Smith, object: Ryan Reynolds
2025-08-07 16:25:37 __main__ INFO     patch_obj_idx=1 | ['Ryan Reynolds', 'Ted Cruz', 'James Patterson', 'Gabriel Iglesias', 'Paulo Dybala', 'Diane von Furstenberg']
2025-08-07 16:25:37 __main__ INFO     clean_obj_idx=0 | ['Ryan Reynolds', 'Ted Cruz', 'James Patterson', 'Gabriel Iglesias', 'Paulo Dybala', 'Diane von Furstenberg']


In [14]:
from src.functional import generate_with_patch

for sample in [patch_sample, clean_sample]:
    print(sample.prompt(), ">>", sample.obj)
    attn_pattern = verify_head_patterns(
        prompt=sample.prompt(),
        options=sample.options,
        pivot=sample.subj,
        mt=mt,
        heads=HEADS,
        generate_full_answer=True
    )

1. Ryan Reynolds
2. Ted Cruz
3. James Patterson
4. Gabriel Iglesias
5. Paulo Dybala
6. Diane von Furstenberg
Who among these people mentioned above is a politician by profession?
Answer: >> Ted Cruz
0 patches to ablate possible answer information from options
2025-08-07 16:25:41 src.experiments.utils DEBUG    Generated full answer: " Ted Cruz
The given list includes a variety of professions such as actors, authors, comedians, fashion designers, and athletes. However, the only"
2025-08-07 16:25:41 src.experiments.utils DEBUG    Predictions: ['" Ted"[23989] (p=0.742, logit=21.500)', '" "[220] (p=0.069, logit=19.125)', '" ("[320] (p=0.061, logit=19.000)', '" The"[578] (p=0.029, logit=18.250)', '" Option"[7104] (p=0.022, logit=18.000)']
2025-08-07 16:25:41 src.experiments.utils INFO     Combined attention matrix for all heads


1. Ryan Reynolds
2. Ted Cruz
3. James Patterson
4. Gabriel Iglesias
5. Paulo Dybala
6. Diane von Furstenberg
Who among these people mentioned above is a actor by profession?
Answer: >> Ryan Reynolds
0 patches to ablate possible answer information from options
2025-08-07 16:25:45 src.experiments.utils DEBUG    Generated full answer: " Ryan Reynolds and Gabriel Iglesias are actors by profession. Ryan Reynolds is a Canadian actor, film producer, and screenwriter, while Gabriel Igles"
2025-08-07 16:25:45 src.experiments.utils DEBUG    Predictions: ['" Ryan"[13960] (p=0.602, logit=20.000)', '" Among"[22395] (p=0.056, logit=17.625)', '" Option"[7104] (p=0.049, logit=17.500)', '" "[220] (p=0.043, logit=17.375)', '" The"[578] (p=0.043, logit=17.375)']
2025-08-07 16:25:45 src.experiments.utils INFO     Combined attention matrix for all heads


In [15]:
mt._model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 8192)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (up_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (down_proj): Linear(in_features=28672, out_features=8192, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((8192,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((8192,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((8192,), eps=1e-05)
    (rotary_

In [34]:
import baukit
from src.functional import get_module_nnsight, PatchSpec
from src.hooking.llama_attention import LlamaAttentionPatcher
import types
from typing import Literal
from src.tokens import prepare_input


def set_attn_implementation(mt, attn_implementation: Literal["sdpa", "eager"]):
    mt.config._attn_implementation = attn_implementation
    for layer_idx in range(mt.config.num_hidden_layers):
        attn_block_name = mt.attn_module_name_format.format(layer_idx)
        attn_block = baukit.get_module(mt._model, attn_block_name)
        attn_block.config._attn_implementation = attn_implementation


###################################################################################
batch_size = 1  # tokenized.input_ids.shape[0]
n_heads = mt.config.num_attention_heads
head_dim = mt.n_embd // n_heads
query_idx = -1 # almost always the last token
###################################################################################

mt.reset_forward()
set_attn_implementation(mt, "sdpa")

layer_idx, head_idx = HEADS[0]

attn_block_name = mt.attn_module_name_format.format(layer_idx)
attn_block = baukit.get_module(mt._model, attn_block_name)
attn_block.forward = types.MethodType(
    LlamaAttentionPatcher(block_name=attn_block_name),
    attn_block,
)

patch_tokenized = prepare_input(prompts=patch_sample.prompt(), tokenizer=mt)
patch_seq_len = patch_tokenized.input_ids.shape[1]
input_ln = mt.layer_name_format.format(layer_idx) + ".input_layernorm"

with mt.trace(patch_tokenized) as trace:
    ln_module = get_module_nnsight(mt, input_ln)
    patch_ln = ln_module.output.save()

    q_proj_name = mt.attn_module_name_format.format(layer_idx) + ".q_proj"
    q_proj_module = get_module_nnsight(mt, q_proj_name)
    patch_q_proj = q_proj_module.output.view(batch_size, patch_seq_len, n_heads, head_dim).transpose(1, 2).save()
    # patch_q_proj = PatchSpec(
    #     location=(q_proj_name + f".{head_idx}", -1),
    #     patch=patch_q_proj[:, head_idx, query_idx, :].squeeze().save()
    # )

clean_tokenized = prepare_input(prompts=clean_sample.prompt(), tokenizer=mt)
clean_seq_len = clean_tokenized.input_ids.shape[1]
with mt.trace(clean_tokenized) as trace:
    ln_module = get_module_nnsight(mt, input_ln)
    clean_ln = ln_module.output.save()

    q_proj_name = mt.attn_module_name_format.format(layer_idx) + ".q_proj"
    q_proj_module = get_module_nnsight(mt, q_proj_name)
    clean_q_proj = q_proj_module.output.view(batch_size, clean_seq_len, n_heads, head_dim).transpose(1, 2).save()
    # clean_q_proj = PatchSpec(
    #     location=(q_proj_name + f".{head_idx}", -1),
    #     patch=clean_q_proj[:, head_idx, query_idx, :].squeeze().save()
    # )

mt.reset_forward()
set_attn_implementation(mt, "eager")

patch_q_proj.shape, clean_q_proj.shape, patch_ln.shape, clean_ln.shape

2025-08-07 16:26:54 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.33.self_attn
2025-08-07 16:26:54 src.hooking.llama_attention DEBUG    hidden_shape=(1, 52, -1, 128) | input_shape=torch.Size([1, 52]) | torch.Size([1, 52, 8192])
2025-08-07 16:26:54 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 52, 128]) | key_states.size()=torch.Size([1, 8, 52, 128]) | value_states.size()=torch.Size([1, 8, 52, 128])
2025-08-07 16:26:55 src.hooking.llama_attention DEBUG    LlamaAttentionPatcher <> model.layers.33.self_attn
2025-08-07 16:26:55 src.hooking.llama_attention DEBUG    hidden_shape=(1, 52, -1, 128) | input_shape=torch.Size([1, 52]) | torch.Size([1, 52, 8192])
2025-08-07 16:26:55 src.hooking.llama_attention DEBUG    query_states.size()=torch.Size([1, 64, 52, 128]) | key_states.size()=torch.Size([1, 8, 52, 128]) | value_states.size()=torch.Size([1, 8, 52, 128])


(torch.Size([1, 64, 52, 128]),
 torch.Size([1, 64, 52, 128]),
 torch.Size([1, 52, 8192]),
 torch.Size([1, 52, 8192]))

In [35]:
# print(patch_sample.prompt())

In [36]:
# print(clean_sample.prompt())

In [37]:
torch.allclose(
    patch_q_proj[:, head_idx, query_idx, :], 
    clean_q_proj[:, head_idx, query_idx, :],
    atol=1e-3
)

False

In [38]:
torch.allclose(
    patch_ln[:, query_idx, :], 
    clean_ln[:, query_idx, :],
    atol=1e-3
)

False

In [39]:
patch_ln[:, query_idx, :], clean_ln[:, query_idx, :]

(tensor([[ 0.0544, -0.0879,  0.0996,  ..., -0.1147,  0.4746,  0.0728]],
        device='cuda:3', dtype=torch.bfloat16, grad_fn=<SliceBackward0>),
 tensor([[ 0.0056, -0.1758,  0.0693,  ..., -0.0752,  0.4062,  0.0608]],
        device='cuda:3', dtype=torch.bfloat16, grad_fn=<SliceBackward0>))

In [40]:
patch_ln.shape

torch.Size([1, 52, 8192])

In [41]:
# manual calculation
attn_module = baukit.get_module(mt._model, mt.attn_module_name_format.format(layer_idx))
patch_q_proj_manual = attn_module.q_proj(patch_ln)
clean_q_proj_manual = attn_module.q_proj(clean_ln)

print(patch_q_proj_manual.shape, clean_q_proj_manual.shape)
print(torch.allclose(
    patch_q_proj_manual[:, query_idx, :], 
    clean_q_proj_manual[:, query_idx, :],
    atol=1e-3
))

patch_q_proj_manual = patch_q_proj_manual.reshape(batch_size, patch_seq_len, n_heads, head_dim).transpose(1, 2)
clean_q_proj_manual = clean_q_proj_manual.reshape(batch_size, clean_seq_len, n_heads, head_dim).transpose(1, 2)
print(patch_q_proj_manual.shape, clean_q_proj_manual.shape)

for idx in range(n_heads):
    print(head_idx, torch.allclose(
        patch_q_proj_manual[:, idx, query_idx, :], 
        clean_q_proj_manual[:, idx, query_idx, :],
        atol=1e-3
    ))

torch.Size([1, 52, 8192]) torch.Size([1, 52, 8192])
False
torch.Size([1, 64, 52, 128]) torch.Size([1, 64, 52, 128])
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False
45 False


In [63]:
torch.allclose(
    patch_q_proj_manual[:, head_idx, query_idx, :], 
    patch_q_proj[:, head_idx, query_idx, :],
    atol=1e-3
)

True

In [64]:
replace_q_proj = clean_q_proj.clone()
replace_q_proj[:, head_idx, query_idx, :] = patch_q_proj[:, head_idx, query_idx, :]

print(torch.allclose(
        replace_q_proj[:, head_idx, query_idx, :],
        clean_q_proj[:, head_idx, query_idx, :],
        atol=1e-3
    )
)
print(replace_q_proj.shape)

replace_q_proj = replace_q_proj.transpose(1, 2).reshape(batch_size, clean_seq_len, -1)

rep_patch = PatchSpec(
    location=(q_proj_name, -1),
    patch=replace_q_proj[:, -1, :].squeeze(),
)

ln_patch = PatchSpec(
    location=(input_ln, -1),
    patch=patch_ln[:, query_idx, :].squeeze(),
)

head_q_patch = PatchSpec(
    location=(q_proj_name, head_idx, -1),
    patch=patch_q_proj[:, head_idx, query_idx, :].squeeze(),
)

head_q_patch.location, rep_patch.location

False
torch.Size([1, 64, 52, 128])


(('model.layers.33.self_attn.q_proj', 45, -1),
 ('model.layers.33.self_attn.q_proj', -1))

In [65]:
# clean_q_proj_rs = clean_q_proj.view(batch_size, clean_seq_len, -1)
# patch_q_proj_rs = patch_q_proj.view(batch_size, clean_seq_len, -1)
# clean_q_proj_rs[:, -1, :].shape, patch_q_proj_rs[:, -1, :].shape

In [66]:
# torch.allclose(
#     replace_q_proj[:, -1, :].squeeze(), 
#     # patch_q_proj_rs[:, -1, :].squeeze(),
#     clean_q_proj_rs[:, -1, :].squeeze(), 
#     atol=1e-3
# )

In [67]:
patch_q_proj.shape, clean_q_proj.shape
# torch.allclose(patch_q_proj.patch, clean_q_proj.patch)

(torch.Size([1, 64, 52, 128]), torch.Size([1, 64, 52, 128]))

In [68]:
from src.attention import get_attention_matrices, visualize_attn_matrix
from src.functional import interpret_logits

clean_tokenized = prepare_input(prompts=clean_sample.prompt(), tokenizer=mt)

attn_info = get_attention_matrices(
    input=clean_tokenized,
    mt=mt,
)

attn_matrix = attn_info.attention_matrices[layer_idx, head_idx].squeeze()
visualize_attn_matrix(
    attn_matrix=attn_matrix,
    tokens=[mt.tokenizer.decode(t) for t in clean_tokenized.input_ids[0]],
    q_index=-1,
)

interpret_logits(
    tokenizer=mt,
    logits=attn_info.logits,
)

[PredictedToken(token=' Ryan', prob=0.6015625, logit=20.0, token_id=13960, metadata=None),
 PredictedToken(token=' Among', prob=0.055908203125, logit=17.625, token_id=22395, metadata=None),
 PredictedToken(token=' Option', prob=0.04931640625, logit=17.5, token_id=7104, metadata=None),
 PredictedToken(token=' ', prob=0.04345703125, logit=17.375, token_id=220, metadata=None),
 PredictedToken(token=' The', prob=0.04345703125, logit=17.375, token_id=578, metadata=None)]

In [69]:
from src.attention import get_attention_matrices, visualize_attn_matrix

clean_tokenized = prepare_input(prompts=clean_sample.prompt(), tokenizer=mt)

patched_attn_info = get_attention_matrices(
    input=clean_tokenized,
    mt=mt,
    patches=[ln_patch],
    # patches = [head_q_patch]
    # patches = [rep_patch]
)

patched_attn_matrix = patched_attn_info.attention_matrices[layer_idx, head_idx].squeeze()
visualize_attn_matrix(
    attn_matrix=patched_attn_matrix,
    tokens=[mt.tokenizer.decode(t) for t in clean_tokenized.input_ids[0]],
    q_index=-1,
)

interpret_logits(
    tokenizer=mt,
    logits=patched_attn_info.logits,
)

model.layers.33.input_layernorm -1 None replace
is working
log torch.Size([1, 52, 8192]) ('model.layers.33.input_layernorm', -1) torch.Size([8192])
>> false


[PredictedToken(token=' Ryan', prob=0.63671875, logit=20.125, token_id=13960, metadata=None),
 PredictedToken(token=' Among', prob=0.046142578125, logit=17.5, token_id=22395, metadata=None),
 PredictedToken(token=' Option', prob=0.046142578125, logit=17.5, token_id=7104, metadata=None),
 PredictedToken(token=' ', prob=0.040771484375, logit=17.375, token_id=220, metadata=None),
 PredictedToken(token=' (', prob=0.040771484375, logit=17.375, token_id=320, metadata=None)]

In [73]:
for patch in [ln_patch, head_q_patch, rep_patch]:
    print(patch.location, patch.patch.shape)

('model.layers.33.input_layernorm', -1) torch.Size([8192])
('model.layers.33.self_attn.q_proj', 45, -1) torch.Size([128])
('model.layers.33.self_attn.q_proj', -1) torch.Size([8192])


In [77]:
# rep_patch.location = ln_patch.location #! the problem is with the location, not the patch

In [78]:
# from src.attention import get_attention_matrices, visualize_attn_matrix

clean_tokenized = prepare_input(prompts=clean_sample.prompt(), tokenizer=mt)

patched_attn_info = get_attention_matrices(
    input=clean_tokenized,
    mt=mt,
    # patches = [head_q_patch]
    patches = [rep_patch]
)

patched_attn_matrix = patched_attn_info.attention_matrices[layer_idx, head_idx].squeeze()
visualize_attn_matrix(
    attn_matrix=patched_attn_matrix,
    tokens=[mt.tokenizer.decode(t) for t in clean_tokenized.input_ids[0]],
    q_index=-1,
)

interpret_logits(
    tokenizer=mt,
    logits=patched_attn_info.logits,
)

model.layers.33.input_layernorm -1 None replace
is working
log torch.Size([1, 52, 8192]) ('model.layers.33.input_layernorm', -1) torch.Size([8192])
>> false


[PredictedToken(token=' Ryan', prob=0.57421875, logit=18.5, token_id=13960, metadata=None),
 PredictedToken(token=' Ted', prob=0.07763671875, logit=16.5, token_id=23989, metadata=None),
 PredictedToken(token=' (', prob=0.047119140625, logit=16.0, token_id=320, metadata=None),
 PredictedToken(token=' ', prob=0.03662109375, logit=15.75, token_id=220, metadata=None),
 PredictedToken(token=' The', prob=0.032470703125, logit=15.625, token_id=578, metadata=None)]